In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

## Batch

모델을 학습하기 위해 encoder-decoder 데이터를 준비하는 클래스임.

**Batch:**

``` python
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()

           ...
```
`src_mask`:  
pad가 '0'일 때 mask값은 0임.  
encoder의 attention에서는 mask == 0 일 때,  
scores = scores.masked_fill(mask == 0, -1e9)  

`trg :`
여기서 trg는 일반적으로 목표 문장(타겟 시퀀스) 을 의미하고,  
기계번역이나 시퀀스 생성 모델에서는 다음과 같은 두 가지 역할로 나뉘어 사용됩니다:  

`decoder input`: 이전 단어까지의 시퀀스를 입력으로 사용  
`target output`: 실제 정답 시퀀스 (예측 대상)  

`trg[:, :-1]의 의미`  
trg[:, :-1]은 타겟 시퀀스의 마지막 토큰을 잘라낸 것입니다.  
예를 들어 trg가 다음과 같다면:  

trg = [[BOS, A, B, C, EOS]]  
여기서:  

BOS: Begin of Sentence  
EOS: End of Sentence  
trg[:, :-1]은 다음과 같은 결과를 만듭니다:  

`[[BOS, A, B, C]]`  

`디코더의 입력:`  
왜 이렇게 하나요?  
시퀀스 투 시퀀스 모델(예: Transformer, LSTM 기반 NMT)은 일반적으로 디코더에 이전 단어들을 입력으로 주고,  
다음 단어를 예측합니다.  

입력 시퀀스: [BOS, A, B, C] → 디코더에 들어감  
예측 대상: [A, B, C, EOS] → 모델이 맞혀야 할 정답  

  

|디코더 입력 (trg[:, :-1])|	예측 대상 (trg[:, 1:]) | 
|---------------------------|----------------------|
|[BOS, A, B, C]	| [A, B, C, EOS]                  | 

이렇게 나누어야 디코더가 한 단계씩 다음 토큰을 예측할 수 있어요.  

`요약:`  
self.trg = trg[:, :-1]는 디코더 입력을 만들기 위한 코드입니다.  
타겟 시퀀스에서 마지막 토큰(EOS 등)을 제거해 이전 단어까지만 남기는 역할을 합니다.  
이렇게 전처리해야 디코더가 시퀀스를 한 단계씩 예측할 수 있게 됩니다.  



`self.trg_mask 분석해봄`:
---

``` python
self.trg_mask = self.make_std_mask(self.trg, pad)
```

디코더의 self.attention에서 사용할 마스크(mask)를 생성할 코드임.  
  
**목적 요약**: 
- Padding 토큰 무시하기  
→ trg 안의 pad 토큰은 무시되어야 함 (학습에 영향을 주면 안 됨) 

- 미래 단어 가리기 (Look-ahead masking)  
→ 현재 시점보다 미래 단어는 보지 못하게 막기 (언어 모델처럼 auto-regressive하게 학습하기 위함)


``` python
def make_std_mask(tgt, pad):
    # padding이 아닌 부분은 True
    tgt_mask = (tgt != pad).unsqueeze(-2)  # shape: [batch, 1, seq_len]

    # look-ahead mask 추가 (미래 단어 보지 못하게)
    tgt_mask = tgt_mask & Variable(
        subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))  # shape: [1, seq_len, seq_len]

    return tgt_mask
```

`subsequent_mask(tgt.size(-1))`  
이 함수는 하삼각 행렬(triangular matrix) 형태의 마스크를 만듭니다.  
시점 t에서는 t보다 이후 시점 단어를 보지 못하게끔 만듭니다.  
예: 길이 5인 문장이라면  
```
[[1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [1, 1, 1, 0, 0],
 [1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1]]
```

``` python
 tgt_mask = tgt_mask & Variable(
        subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))  # shape: [1, seq_len, seq_len]
```

이렇게 하면 pad_mask == 0 부분을 값도 false로 되어 soft_mask값에 반영되지 않음.  
여기서, tgt_mask는 broadcasting 되어 연산됨.
예를 들면:  
``` python
m = subsequent_mask(3)
t = torch.tensor([[[False,True, True]]])
print(t&m)
```
결과는 :
tensor([[[False, False, False],
         [False,  True, False],
         [False,  True,  True]]])

In [5]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

#batch: number of batch
#nbatches : nuber of batch run.
#V : num of vacabulary
def data_gen(V, batch, nbatches, seq_len=10):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, seq_len)))
        #시작 토큰(Start Token)을 고정하는 역할
        data[:, 0] = 1
        src = Variable(data, requires_grad=False)
        tgt = Variable(data, requires_grad=False)
        yield Batch(src, tgt, 0)
    
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        #print('tgt_mask.shape', tgt_mask.shape)
        #print('tgt_mask', tgt_mask)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

### Batch 클래스 사용법

In [7]:
nV = 10 #dim of vocabulary
seq_len = 10
batch = 30
nbatches = 20
data = torch.from_numpy(np.random.randint(1, nV, size=(batch, seq_len)))
#시작 토큰(Start Token)을 고정하는 역할
data[:, 0] = 1
src = Variable(data, requires_grad=False)
tgt = Variable(data, requires_grad=False)
inputs =  Batch(src, tgt, 0)

tgt_mask.shape torch.Size([30, 1, 9])
tgt_mask tensor([[[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True]],

        [[True,

`data_gen` 함수는 V 크기의 단어로 이루어진, batch 크기의 입출력 데이터를  
`n_batches` 횟수만큼 발생시킴

In [8]:
data_iters = data_gen(nV,batch,nbatches,seq_len)